# Ciminion2 DRL Gröbner Basis
DRL Gröbner basis computation for an arbitrary Ciminion instance.

In [1]:
load("Ciminion_2.sage")
load("Ciminion_2_polynomial_model.sage")

## Ciminion Instance

In [2]:
ciminion_2 = Ciminion_2(info_level=1)

Ciminion_2 parameters
Field: Finite Field of size 170141183460469231731687303715884105773
Rounds_C: 90
Rounds_E: 14
Constants_C: [[55849192389615766811261520101276943678, 99048917482815628458284176042138594856, 9562083465749324334991598936301718022, 108342433898288718447305527714005668611], [101610103479678492662519135503710710103, 74718989151249692721773209250918143875, 30715098658186954932844762142223052387, 125849874306588208237750970053729087218], [67687457516589494895652870710917703925, 1293475201257373232068872453813031714, 35918186205146122149685073307171756939, 83791661272413626297206640062545085199], [95422863601950854497887513835755050220, 61985591982940264036661198586177515474, 2143791136902223620089036809234452166, 159821631587278282370866955457792751981], [123262742816565641546235362787827574650, 71158935765824562632686398925633012791, 128201559307602656567321220711468803761, 63388917294851002631597979447528461676], [73163091404355328414065424228688471359, 3537963357177870

## Ciminion Polynomial model

In [3]:
polys = generate_Ciminion_2_polynomials(ciminion_2=ciminion_2)

P = polys[0].parent()

Plaintext: [139297645977815968818393349075248207130, 78772067618645615985966881370884907956]
Key: [156918828523651136501213434265323453292, 27365218297115032785116309524632126390]
Nonce: 1177172959307210492343091257332370352
Ciphertext: [35279699061002023204907924679223355723, 104911018700299778444127352413522622003]
Term order: degrevlex


## DRL Gröbner Basis computation
First we change to a different DRL term order.

In [4]:
f = efficient_Ciminion_2_termorder(ciminion_2, P)

Next we invert the matrices in the round functions, extract the linear polynomials in each round, and perform Gaussian elimination on the linear polynomials.

In [5]:
polys = [f(poly) for poly in transform_Ciminion_2_polynomial_system(ciminion_2, polys)]

lin_polys = Sequence([poly for poly in polys if poly.degree() == 1])
M, v = lin_polys.coefficients_monomials()

M = M.echelon_form()

lin_polys = ideal((M * v).list())

Now we reduce the non-linear polynomials with respect to the linear polynomials, i.e. we eliminate variables in the non-linear polynomials.
We call the resulting polynomial system the downsized Gröbner basis.

In [6]:
gb_downsized = [poly.reduce(lin_polys) for poly in polys if poly.degree() > 1]
lms = [poly.lm() for poly in gb_downsized]

In [7]:
for mon in lms:
    print(mon)

x_C_3__2^2
x_C_3__3^2
x_C_3__4^2
x_C_3__5^2
x_C_3__6^2
x_C_3__7^2
x_C_3__8^2
x_C_3__9^2
x_C_3__10^2
x_C_3__11^2
x_C_3__12^2
x_C_3__13^2
x_C_3__14^2
x_C_3__15^2
x_C_3__16^2
x_C_3__17^2
x_C_3__18^2
x_C_3__19^2
x_C_3__20^2
x_C_3__21^2
x_C_3__22^2
x_C_3__23^2
x_C_3__24^2
x_C_3__25^2
x_C_3__26^2
x_C_3__27^2
x_C_3__28^2
x_C_3__29^2
x_C_3__30^2
x_C_3__31^2
x_C_3__32^2
x_C_3__33^2
x_C_3__34^2
x_C_3__35^2
x_C_3__36^2
x_C_3__37^2
x_C_3__38^2
x_C_3__39^2
x_C_3__40^2
x_C_3__41^2
x_C_3__42^2
x_C_3__43^2
x_C_3__44^2
x_C_3__45^2
x_C_3__46^2
x_C_3__47^2
x_C_3__48^2
x_C_3__49^2
x_C_3__50^2
x_C_3__51^2
x_C_3__52^2
x_C_3__53^2
x_C_3__54^2
x_C_3__55^2
x_C_3__56^2
x_C_3__57^2
x_C_3__58^2
x_C_3__59^2
x_C_3__60^2
x_C_3__61^2
x_C_3__62^2
x_C_3__63^2
x_C_3__64^2
x_C_3__65^2
x_C_3__66^2
x_C_3__67^2
x_C_3__68^2
x_C_3__69^2
x_C_3__70^2
x_C_3__71^2
x_C_3__72^2
x_C_3__73^2
x_C_3__74^2
x_C_3__75^2
x_C_3__76^2
x_C_3__77^2
x_C_3__78^2
x_C_3__79^2
x_C_3__80^2
x_C_3__81^2
x_C_3__82^2
x_C_3__83^2
x_C_3__84^2
x_C_3__85^2


In [8]:
gcds = [gcd(comb[0], comb[1]) for comb in Combinations(lms, 2)]
list(set(gcds))

[1]

Since all leading monomials are pairwise coprime we have indeed constructed a DRL Gröbner basis.

Finally, let us verify that the downsized Gröbner basis is indeed zero_dimensional by extracting the variables present in the downsized system.

In [9]:
P = gb_downsized[0].parent()
variables = P.gens()
eliminated_variables = [poly.lm() for poly in lin_polys.gens()]
variables_downsized = [var for var in variables if not var in eliminated_variables]

In [10]:
ideal([var**2 for var in variables_downsized]) == ideal(lms)

True

In [11]:
M, v = Sequence(gb_downsized).coefficients_monomials()
M = M.echelon_form()

gb_downsized_red = list((vector(M * v)))

In [12]:
sep = 100 * "-"

for poly in gb_downsized_red:
    print(poly)
    print(sep)

x_C_3__2^2 + 86863947761142757040929671647727363139*x_C_3__2*x_C_3__3 + 9991288321016978996336223940288100074*x_C_3__2*x_C_3__4 + 103679979676889098356366292683173366157*x_C_3__2*x_C_3__5 + 121734911939993348855457300921701347837*x_C_3__2*x_C_3__6 + 46787888200552555418875618696567697419*x_C_3__2*x_C_3__7 + 118604103914600678352185059893748526592*x_C_3__2*x_C_3__8 + 56359809289922196630646733165543535427*x_C_3__2*x_C_3__9 + 14908870629609406364744108534941567980*x_C_3__2*x_C_3__10 + 133324598705730086027581412168914957510*x_C_3__2*x_C_3__11 + 94044235997027518865460147570526024881*x_C_3__2*x_C_3__12 + 72395219107126337475472650616758181791*x_C_3__2*x_C_3__13 + 59031787639211590868443556980610924906*x_C_3__2*x_C_3__14 + 112026277951842000648759426032996214490*x_C_3__2*x_C_3__15 + 115577243030601992489174751816820452732*x_C_3__2*x_C_3__16 + 6083723541697113347210980229733046704*x_C_3__2*x_C_3__17 + 117628157656650571161974867221893433906*x_C_3__2*x_C_3__18 + 10053400981659221352428480837

In [13]:
set(flatten([poly.variables() for poly in gb_downsized_red])) == set(flatten([poly.lm().variables() for poly in gb_downsized_red]))

True